# Example of data access via the s3 bucket and data calibration

Example of data access and calibration of P-band SAR data on the NASA MAAP.

Load the libraries

In [ ]:
from osgeo import gdal
from gdalconst import GA_ReadOnly
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sg
import sys
sys.path.insert(0, '/projects/Scripts')

# Increase figure size (can be modified for bigger or smaller figures):
plt.rcParams["figure.figsize"] = 20,20

## 1. Open a P-band SAR image

Open SAR image (HV polarisation) in slant range geometry

In [ ]:
inputFilename = '/projects/s3-drive/catalog-data/Campaign_data/afrisar_dlr/afrisar_dlr_T2-0_SLC_HV.tiff'
input_image_driver = gdal.Open(inputFilename, GA_ReadOnly)
input_image = input_image_driver.ReadAsArray()
RasterXSize = input_image_driver.RasterXSize
RasterYSize = input_image_driver.RasterYSize
input_image_driver = None

Display SAR image in slant range geometry

In [ ]:
imgplot = plt.imshow(np.absolute(input_image))

## 2. Open DEM and compute local incidence angle

Define SAR image parameters 

In [ ]:
pixel_spacing_rg = 1.1988876
z_flight = 6383.36
z_terrain = 287.50
SLR_start = 6536.3352

Project DEM from ground-range to slant-range (sensor) geometry

In [ ]:
demGrdFile = '/projects/s3-drive/catalog-data/Campaign_data/afrisar_dlr/afrisar_dlr_dem_S_T2-0.tiff'
demSlrFile = '/projects/dem_SR.tiff'
lutFile = '/projects/s3-drive/catalog-data/Campaign_data/afrisar_dlr/afrisar_dlr_T2-0_lut.tiff'

import projectors
projectors.GrdToSlrProj(demGrdFile, demSlrFile, lutFile, inputFilename)

Open DEM in slant range geometry

In [ ]:
dem_driver = gdal.Open(demSlrFile, GA_ReadOnly)
dem = dem_driver.ReadAsArray()
dem_driver = None

Initialise structures and compute of local slope

In [ ]:
angle1 = np.full((RasterYSize, RasterXSize), np.NaN, dtype=dem.dtype)
angle2 = np.full((RasterYSize, RasterXSize), np.NaN, dtype=dem.dtype)

angle1[:, 1:-1] = np.arctan2(dem[:, 1:-1] - dem[:, :-2], pixel_spacing_rg)
angle2[:, 1:-1] = np.arctan2(dem[:, 2:] - dem[:, 1:-1], pixel_spacing_rg)
angle = (angle1 + angle2)/2

Filter angle map (boxcarFilter 5x5)

In [ ]:
size = [5, 5]
angle = sg.convolve2d(angle, np.ones(size), 'same') / (size[0] * size[1])

Local incidence angle computation

In [ ]:
theta_inc_map = np.abs(np.arccos((z_flight - z_terrain) / (SLR_start + np.mgrid[:RasterYSize, :RasterXSize][1] * pixel_spacing_rg)) - angle)

Display incidence angle map in slant range geometry

In [ ]:
imgplot = plt.imshow(theta_inc_map)

## 3. Calibrate SAR image

Calibration of the SAR data to Sigma0 (natural)

In [ ]:
sigma0 = np.absolute(input_image)**2 * np.sin(theta_inc_map)

# Close dataset to save memory:
theta_inc_map = None

# Filter bad data:
sigma0[sigma0 <= 0] = np.NaN

Example of basic multilooking: 5x5 boxcar filter (can be changed to increase number of looks)

In [ ]:
size = [5, 5]
sigma0 = sg.convolve2d(sigma0, np.ones(size), 'same') / (size[0] * size[1])

Convert to dB and set a threshold to -45 dB (arbitrary value)

In [ ]:
sigma0 = 10 * np.log10(sigma0)
sigma0[sigma0 < -45] = -45

Display sigma0 in slant range geometry

In [ ]:
imgplot = plt.imshow(sigma0)

Save calibrated sigma0 in slant-range geometry

In [ ]:
slrFile = '/projects/sigma0.tiff'

# Save output image in slant range geometry:
outdriver = gdal.GetDriverByName('GTiff')
output_image_driver = outdriver.Create(slrFile, RasterXSize, RasterYSize, 1, gdal.GDT_Float32)
output_image_driver.GetRasterBand(1).WriteArray(sigma0)
output_image_driver = None